# Experiment Sarimax

In [1]:
!pip install kaggle wandb onnx -Uq
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 53.2 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
! unzip walmart-recruiting-store-sales-forecasting.zip
!unzip train.csv.zip
!unzip features.csv.zip

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 474MB/s]
Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           
Archive:  train.csv.zip
  inflating: train.csv               
Archive:  features.csv.zip
  inflating: features.csv            


In [3]:
!pip install statsmodels mlflow dagshub scikit-learn pandas numpy matplotlib seaborn joblib -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 741.4/741.4 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

In [4]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Core libraries
import mlflow
import mlflow.sklearn
import dagshub
from datetime import datetime, timedelta
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import joblib
import os

# SARIMAX and time series
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings
import logging
logging.getLogger('statsmodels').setLevel(logging.WARNING)


In [5]:
# Additional warning suppression for cleaner output
import warnings
warnings.filterwarnings('ignore', message='No frequency information was provided')
warnings.filterwarnings('ignore', message='An unsupported index was provided')
warnings.filterwarnings('ignore', message='A date index has been provided, but it has no associated frequency')
warnings.filterwarnings('ignore', message='No supported index is available')
warnings.filterwarnings('ignore', category=FutureWarning, module='statsmodels')
warnings.filterwarnings('ignore', message='Maximum Likelihood optimization failed to converge')
warnings.filterwarnings('ignore', category=UserWarning, module='statsmodels')



In [10]:
class WalmartPreprocessingPipeline:
    """
    Complete preprocessing pipeline for Walmart sales data - EXACT COPY from experiment_11_sarimax.ipynb
    Supports fit/transform pattern for proper train/validation handling
    """

    def __init__(self):
        self.fitted = False
        self.outlier_thresholds = None
        self.feature_columns = None

    def load_and_prepare_data(self):
        """Load and merge train.csv, stores.csv, features.csv datasets"""
        print("📊 Loading datasets...")

        # Load datasets
        train_df = pd.read_csv('train.csv')
        stores_df = pd.read_csv('stores.csv')
        features_df = pd.read_csv('features.csv')

        print(f"   📈 Train data: {train_df.shape}")
        print(f"   🏪 Stores data: {stores_df.shape}")
        print(f"   🎯 Features data: {features_df.shape}")

        # Convert Date column to datetime
        train_df['Date'] = pd.to_datetime(train_df['Date'])
        features_df['Date'] = pd.to_datetime(features_df['Date'])

        # Merge datasets
        train_stores = train_df.merge(stores_df, on='Store', how='left')
        train_full = train_stores.merge(features_df, on=['Store', 'Date'], how='left')

        print(f"   ✅ Merged data: {train_full.shape}")
        print(f"   📅 Date range: {train_full['Date'].min()} to {train_full['Date'].max()}")

        return train_full

    def clean_merged_data(self, train_full):
        """Clean merged data by handling duplicate IsHoliday columns"""
        print("🧹 Cleaning merged data...")

        initial_shape = train_full.shape

        # Handle duplicate IsHoliday columns if they exist
        if 'IsHoliday_x' in train_full.columns and 'IsHoliday_y' in train_full.columns:
            print("   🔄 Resolving duplicate IsHoliday columns...")
            train_full['IsHoliday'] = train_full['IsHoliday_x'] | train_full['IsHoliday_y']
            train_full = train_full.drop(['IsHoliday_x', 'IsHoliday_y'], axis=1)

        print(f"   ✅ Cleaned data: {train_full.shape} (was {initial_shape})")
        return train_full

    def create_temporal_split(self, df, train_ratio=0.8):
        """Create temporal split to prevent data leakage"""
        print(f"📅 Creating temporal split ({int(train_ratio*100)}/{int((1-train_ratio)*100)})...")

        # Sort by date to ensure temporal order
        df_sorted = df.sort_values('Date').reset_index(drop=True)

        # Find split point
        split_idx = int(len(df_sorted) * train_ratio)
        split_date = df_sorted.iloc[split_idx]['Date']

        # Create splits
        train_data = df_sorted.iloc[:split_idx].copy()
        val_data = df_sorted.iloc[split_idx:].copy()

        # Create split info dictionary
        split_info = {
            'split_date': split_date,
            'train_size': len(train_data),
            'val_size': len(val_data),
            'train_date_range': (train_data['Date'].min(), train_data['Date'].max()),
            'val_date_range': (val_data['Date'].min(), val_data['Date'].max())
        }

        print(f"   📊 Split date: {split_date}")
        print(f"   📈 Train: {len(train_data):,} records ({train_data['Date'].min()} to {train_data['Date'].max()})")
        print(f"   📉 Val: {len(val_data):,} records ({val_data['Date'].min()} to {val_data['Date'].max()})")

        return train_data, val_data, split_info

    def fit(self, train_data):
        """Fit the preprocessing pipeline on training data"""
        print("🔧 Fitting preprocessing pipeline on training data...")

        # Store training data for lag feature creation
        self.train_data_for_lags = train_data.copy()

        # Fit outlier removal thresholds on training data only - EXACT from experiment_11_sarimax
        self.outlier_thresholds = {
            'A': {'lower': -1000, 'upper': 50000},  # Type A stores
            'B': {'lower': -500, 'upper': 25000},   # Type B stores
            'C': {'lower': -200, 'upper': 15000}    # Type C stores
        }

        print("✅ Pipeline fitted on training data")
        self.fitted = True
        return self

    def transform(self, data, is_validation=False):
        """Transform data using fitted pipeline - EXACT from experiment_11_sarimax"""
        if not self.fitted:
            raise ValueError("Pipeline must be fitted before transform!")

        print(f"🔄 Transforming {'validation' if is_validation else 'training'} data...")

        df = data.copy()

        # Step 1: Create date features
        df = self._create_date_features(df)

        # Step 2: Create holiday features
        df = self._create_holiday_features(df)

        # Step 3: Encode categorical features (BEFORE outlier removal!)
        df = self._encode_categorical_features(df)

        # Step 4: Create lag features (different for train vs validation)
        if is_validation:
            df = self._create_lag_features_validation(df)
        else:
            df = self._create_lag_features_training(df)

        # Step 5: Remove outliers (only on training data, AFTER encoding)
        if not is_validation:
            df = self._remove_outliers(df)

        # Step 6: Remove markdown features
        df = self._remove_markdown_features(df)

        # Step 7: Remove redundant features
        df = self._remove_redundant_features(df)

        print(f"✅ Transform complete. Shape: {df.shape}")
        return df

    def fit_transform(self, train_data):
        """Fit and transform training data in one step"""
        return self.fit(train_data).transform(train_data, is_validation=False)

    def _create_date_features(self, df):
        """Create date features - EXACT from experiment_11_sarimax"""
        df = df.copy()
        df['Year'] = df['Date'].dt.year
        df['Month'] = df['Date'].dt.month
        df['Day'] = df['Date'].dt.day
        df['DayOfWeek'] = df['Date'].dt.dayofweek
        df['WeekOfYear'] = df['Date'].dt.isocalendar().week
        df['Quarter'] = df['Date'].dt.quarter
        df['IsWeekend'] = (df['DayOfWeek'] >= 5).astype(int)
        df['IsMonthStart'] = df['Date'].dt.is_month_start.astype(int)
        df['IsMonthEnd'] = df['Date'].dt.is_month_end.astype(int)
        df['IsQuarterStart'] = df['Date'].dt.is_quarter_start.astype(int)
        df['IsQuarterEnd'] = df['Date'].dt.is_quarter_end.astype(int)
        start_date = df['Date'].min()
        df['DaysFromStart'] = (df['Date'] - start_date).dt.days
        df['WeeksFromStart'] = df['DaysFromStart'] // 7
        return df

    def _create_holiday_features(self, df):
        """Create holiday features - EXACT from experiment_11_sarimax"""
        df = df.copy()
        super_bowl_dates = ['2010-02-12', '2011-02-11', '2012-02-10']
        labor_day_dates = ['2010-09-10', '2011-09-09', '2012-09-07']
        thanksgiving_dates = ['2010-11-26', '2011-11-25', '2012-11-23']
        christmas_dates = ['2010-12-31', '2011-12-30', '2012-12-28']

        df['IsSuperBowlWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(super_bowl_dates).astype(int)
        df['IsLaborDayWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(labor_day_dates).astype(int)
        df['IsThanksgivingWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(thanksgiving_dates).astype(int)
        df['IsChristmasWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(christmas_dates).astype(int)
        df['IsMajorHoliday'] = (df['IsSuperBowlWeek'] | df['IsLaborDayWeek'] |
                               df['IsThanksgivingWeek'] | df['IsChristmasWeek']).astype(int)
        df['IsHolidayMonth'] = df['Month'].isin([11, 12]).astype(int)
        df['IsBackToSchool'] = df['Month'].isin([8, 9]).astype(int)
        return df

    def _create_lag_features_training(self, df):
        """Create lag features for training data - DISABLED from experiment_11_sarimax"""
        # Lag features removed to prevent overfitting
        return df

    def _create_lag_features_validation(self, df):
        """Create lag features for validation data - DISABLED from experiment_11_sarimax"""
        # Lag features removed to prevent overfitting
        return df

    def _remove_outliers(self, df):
        """Remove outliers from training data only - EXACT from experiment_11_sarimax"""
        initial_len = len(df)
        df_clean = df.copy()

        for store_type, thresholds in self.outlier_thresholds.items():
            type_mask = df_clean[f'Type_{store_type}'] == 1
            outlier_mask = (
                (df_clean['Weekly_Sales'] < thresholds['lower']) |
                (df_clean['Weekly_Sales'] > thresholds['upper'])
            )
            df_clean = df_clean[~(type_mask & outlier_mask)]

        removed = initial_len - len(df_clean)
        print(f"   🗑️ Removed {removed:,} outliers from training data")
        return df_clean

    def _remove_markdown_features(self, df):
        """Remove markdown columns - EXACT from experiment_11_sarimax"""
        markdown_cols = [col for col in df.columns if 'MarkDown' in col]
        if markdown_cols:
            df = df.drop(markdown_cols, axis=1)
        return df

    def _remove_redundant_features(self, df):
        """Remove redundant features - EXACT from experiment_11_sarimax"""
        redundant_cols = ['Year', 'Quarter', 'Day', 'WeekOfYear', 'DaysFromStart',
                         'IsQuarterStart', 'IsQuarterEnd']
        existing_redundant = [col for col in redundant_cols if col in df.columns]
        if existing_redundant:
            df = df.drop(existing_redundant, axis=1)
        return df

    def _encode_categorical_features(self, df):
        """Encode categorical features using both one-hot and label encoding - EXACT from experiment_11_sarimax"""
        df = df.copy()

        if 'Type' in df.columns:
            print(f"   🔧 Encoding Type column using both one-hot and label encoding...")

            # One-hot encoding (existing approach)
            type_dummies = pd.get_dummies(df['Type'], prefix='Type', dtype=int)

            # Label encoding (experiment_2 approach)
            # A=0, B=1, C=2 (same as experiment_2)
            type_mapping = {'A': 0, 'B': 1, 'C': 2}
            df['Type_Encoded'] = df['Type'].map(type_mapping)

            # Add one-hot columns
            for col in type_dummies.columns:
                df[col] = type_dummies[col]

            # Remove original Type column
            df = df.drop('Type', axis=1)

            print(f"   ✅ Added both Type_Encoded and {list(type_dummies.columns)}")

        return df


def get_exog_features():
    """Define external regressor features for SARIMAX - EXACT from experiment_11_sarimax"""
    # Use only the most important features to reduce complexity
    return [
        'IsHoliday',
        'Type_Encoded',
        'Month'
    ]


def setup_mlflow():
    """Setup MLflow and DagsHub tracking"""
    print("🔧 Setting up MLflow and DagsHub...")

    try:
        mlflow.end_run()
    except:
        pass

    try:
        dagshub.init(
            repo_owner='konstantine25b',
            repo_name='Walmart-Recruiting---Store-Sales-Forecasting',
            mlflow=True
        )
        print("✅ DagsHub initialized successfully!")
    except Exception as e:
        print(f"⚠️ DagsHub init warning: {e}")

    mlflow.set_tracking_uri("https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow")

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    experiment_name = f"Experiment_SARIMAX_{timestamp}"

    try:
        experiment_id = mlflow.create_experiment(experiment_name)
        print(f"✅ Created new experiment: {experiment_name}")
    except mlflow.exceptions.MlflowException as e:
        if "already exists" in str(e):
            experiment = mlflow.get_experiment_by_name(experiment_name)
            experiment_id = experiment.experiment_id
            print(f"✅ Using existing experiment: {experiment_name}")
        else:
            experiment_name = "Default"
            mlflow.set_experiment(experiment_name)
            print(f"⚠️ Using default experiment due to: {e}")

    mlflow.set_experiment(experiment_name)

    print(f"✅ MLflow setup complete!")
    print(f"🔗 Tracking URI: {mlflow.get_tracking_uri()}")
    print(f"📊 Experiment: {experiment_name}")

    return experiment_name


def get_preprocessed_data():
    """Get model-ready data with external features for SARIMAX - using experiment_11_sarimax pipeline"""
    print("🔄 Getting preprocessed data for SARIMAX...")

    pipeline = WalmartPreprocessingPipeline()
    train_full = pipeline.load_and_prepare_data()
    train_full = pipeline.clean_merged_data(train_full)
    train_data, val_data, split_info = pipeline.create_temporal_split(train_full)

    pipeline.fit(train_data)
    train_processed = pipeline.transform(train_data, is_validation=False)
    val_processed = pipeline.transform(val_data, is_validation=True)

    print(f"✅ SARIMAX data preprocessing complete!")
    print(f"   📊 Training shape: {train_processed.shape}")
    print(f"   📊 Validation shape: {val_processed.shape}")
    print(f"   🎯 External features: {len(get_exog_features())}")

    return train_processed, val_processed, split_info


def calculate_wmae(y_true, y_pred, is_holiday, holiday_weight=5.0):
    """Calculate Weighted Mean Absolute Error (WMAE)"""
    abs_errors = np.abs(y_true - y_pred)
    weights = np.where(is_holiday, holiday_weight, 1.0)
    wmae = np.sum(weights * abs_errors) / np.sum(weights)
    return wmae



In [13]:
def prepare_sarimax_data(train_processed, val_processed):
    """Prepare data for SARIMAX training on Store-Dept combinations"""
    print("📊 Preparing data for SARIMAX modeling...")

    print(f"   📈 Train data shape: {train_processed.shape}")
    print(f"   📉 Val data shape: {val_processed.shape}")

    # Get unique Store-Dept combinations
    train_combinations = set(zip(train_processed['Store'], train_processed['Dept']))
    val_combinations = set(zip(val_processed['Store'], val_processed['Dept']))

    print(f"   🏪 Train combinations: {len(train_combinations)}")
    print(f"   🔮 Val combinations: {len(val_combinations)}")

    # Find missing combinations in validation
    missing_in_val = train_combinations - val_combinations
    missing_in_train = val_combinations - train_combinations

    print(f"   ⚠️ Missing in validation: {len(missing_in_val)}")
    print(f"   ⚠️ Missing in training: {len(missing_in_train)}")

    return train_processed, val_processed, train_combinations, val_combinations


def train_sarimax_models(train_data, val_data):
    """Train ultra-fast SARIMAX(1,1,0)x(0,0,0,0) models with external features"""
    print("📈 Training ultra-fast SARIMAX(1,1,0)x(0,0,0,0) models with external features...")
    print("   ⏰ No time limit - training all combinations")

    train_combinations = train_data.groupby(['Store', 'Dept']).size().index.tolist()
    print(f"   📊 Training models for {len(train_combinations)} combinations")
    print(f"   🎯 Training SARIMAX with {len(get_exog_features())} external features")

    models = {}
    training_errors = {}
    model_orders = {}
    exog_features = get_exog_features()

    successful_models = 0
    failed_models = 0

    for i, (store, dept) in enumerate(train_combinations):
        try:
            store_dept_data = train_data[
                (train_data['Store'] == store) &
                (train_data['Dept'] == dept)
            ].copy()

            if len(store_dept_data) < 10:  # Reduced from 20 for speed
                failed_models += 1
                continue

            store_dept_data = store_dept_data.sort_values('Date')

            # Prepare time series
            ts_data = store_dept_data['Weekly_Sales'].values

            # Prepare external regressors with proper data type handling
            exog_data = None
            if all(feat in store_dept_data.columns for feat in exog_features):
                # Extract external features and ensure numeric types
                exog_df = store_dept_data[exog_features].copy()

                # Convert to numeric, handle any non-numeric values
                for col in exog_features:
                    exog_df[col] = pd.to_numeric(exog_df[col], errors='coerce')

                # Handle missing values with forward fill, then backward fill, then fill with 0
                exog_df = exog_df.fillna(method='ffill').fillna(method='bfill').fillna(0)

                # Convert to numpy array of float64 (required by statsmodels)
                exog_data = exog_df.astype(np.float64).values

                # Check for any remaining non-finite values
                if not np.all(np.isfinite(exog_data)):
                    print(f"   ⚠️ Warning: Non-finite values found in exog data for Store {store}, Dept {dept}")
                    # Replace any remaining non-finite values with 0
                    exog_data = np.nan_to_num(exog_data, nan=0.0, posinf=0.0, neginf=0.0)

            if np.var(ts_data) == 0 or np.std(ts_data) < 1e-6:
                failed_models += 1
                continue

            # Check for non-finite values in time series
            if not np.all(np.isfinite(ts_data)):
                print(f"   ⚠️ Warning: Non-finite values found in time series for Store {store}, Dept {dept}")
                # Skip this combination if time series has non-finite values
                failed_models += 1
                continue

            # EXTREME SPEED: Use ARIMAX instead of SARIMAX (no seasonality)
            sarimax_order = (1, 1, 0)  # Minimal ARIMA: AR(1), I(1), no MA
            seasonal_order = (0, 0, 0, 0)

            # Fit ARIMAX model (SARIMAX with no seasonal component)
            model = SARIMAX(
                ts_data,
                exog=exog_data,
                order=sarimax_order,
                seasonal_order=seasonal_order,
                enforce_stationarity=False,
                enforce_invertibility=False,
                concentrate_scale=True,
                trend='c'  # Just constant trend
            )
            fitted_model = model.fit(
                disp=False,
                maxiter=10,  # Ultra-fast
                method='lbfgs',
                low_memory=True,
                warn_convergence=False
            )

            models[(store, dept)] = fitted_model
            model_orders[(store, dept)] = (sarimax_order, seasonal_order)
            successful_models += 1

            if i % 50 == 0:
                print(f"   ✅ Trained {i+1}/{len(train_combinations)} models ({successful_models} successful, {failed_models} failed)")

        except Exception as e:
            failed_models += 1
            if failed_models < 3:
                print(f"   ⚠️ Failed to train model for Store {store}, Dept {dept}: {e}")

    print(f"✅ SARIMAX training complete!")
    print(f"   🎯 Successful models: {successful_models}")
    print(f"   ❌ Failed models: {failed_models}")
    print(f"   📊 Coverage: {successful_models}/{len(train_combinations)} ({successful_models/len(train_combinations)*100:.1f}%)")

    return models, training_errors, model_orders


def make_sarimax_predictions(models, val_data, train_data=None):
    """Make predictions using trained SARIMAX models with external features"""
    print("📈 Making SARIMAX predictions with external features...")

    predictions = []
    actuals = []
    holidays = []
    successful_predictions = 0
    skipped_predictions = 0
    exog_features = get_exog_features()

    val_combinations = val_data.groupby(['Store', 'Dept']).groups.keys()

    for store, dept in val_combinations:
        try:
            store_dept_val = val_data[
                (val_data['Store'] == store) &
                (val_data['Dept'] == dept)
            ].copy()

            store_dept_val = store_dept_val.sort_values('Date')

            if (store, dept) in models:
                fitted_model = models[(store, dept)]
                n_periods = len(store_dept_val)

                try:
                    # Prepare external regressors for forecasting with proper data type handling
                    exog_forecast = None
                    if all(feat in store_dept_val.columns for feat in exog_features):
                        # Extract external features and ensure numeric types
                        exog_df = store_dept_val[exog_features].copy()

                        # Convert to numeric, handle any non-numeric values
                        for col in exog_features:
                            exog_df[col] = pd.to_numeric(exog_df[col], errors='coerce')

                        # Handle missing values with forward fill, then backward fill, then fill with 0
                        exog_df = exog_df.fillna(method='ffill').fillna(method='bfill').fillna(0)

                        # Convert to numpy array of float64 (required by statsmodels)
                        exog_forecast = exog_df.astype(np.float64).values

                        # Check for any remaining non-finite values
                        if not np.all(np.isfinite(exog_forecast)):
                            # Replace any remaining non-finite values with 0
                            exog_forecast = np.nan_to_num(exog_forecast, nan=0.0, posinf=0.0, neginf=0.0)

                    # Use forecast method with external regressors
                    forecast = fitted_model.forecast(steps=n_periods, exog=exog_forecast)

                    if isinstance(forecast, (int, float)):
                        forecast_list = [forecast] * n_periods
                    elif hasattr(forecast, '__iter__'):
                        forecast_list = list(forecast)
                        if len(forecast_list) < n_periods:
                            last_val = forecast_list[-1] if forecast_list else 0
                            forecast_list.extend([last_val] * (n_periods - len(forecast_list)))
                        elif len(forecast_list) > n_periods:
                            forecast_list = forecast_list[:n_periods]
                    else:
                        skipped_predictions += len(store_dept_val)
                        continue

                    predictions.extend(forecast_list)
                    actuals.extend(store_dept_val['Weekly_Sales'].tolist())
                    holidays.extend(store_dept_val['IsHoliday'].tolist())
                    successful_predictions += len(store_dept_val)

                except Exception as forecast_error:
                    skipped_predictions += len(store_dept_val)
                    continue
            else:
                skipped_predictions += len(store_dept_val)
                continue

        except Exception as e:
            skipped_predictions += len(val_data[
                (val_data['Store'] == store) &
                (val_data['Dept'] == dept)
            ])
            continue

    print(f"✅ SARIMAX predictions complete!")
    print(f"   🎯 SARIMAX predictions: {successful_predictions}")
    print(f"   ⏭️ Skipped (no model): {skipped_predictions}")

    return np.array(predictions), np.array(actuals), np.array(holidays)


def calculate_training_wmae(models, train_data):
    """Calculate training WMAE on fitted values from trained SARIMAX models"""
    print("📊 Calculating training WMAE on fitted values...")

    train_predictions = []
    train_actuals = []
    train_holidays = []

    for (store, dept), fitted_model in models.items():
        try:
            store_dept_data = train_data[
                (train_data['Store'] == store) &
                (train_data['Dept'] == dept)
            ].copy()

            store_dept_data = store_dept_data.sort_values('Date')

            fitted_values = fitted_model.fittedvalues
            actual_values = store_dept_data['Weekly_Sales'].values
            holiday_values = store_dept_data['IsHoliday'].values

            if len(fitted_values) > 0 and len(fitted_values) <= len(actual_values):
                start_idx = len(actual_values) - len(fitted_values)
                aligned_actuals = actual_values[start_idx:]
                aligned_holidays = holiday_values[start_idx:]

                train_predictions.extend(fitted_values.values)
                train_actuals.extend(aligned_actuals)
                train_holidays.extend(aligned_holidays)

        except Exception as e:
            continue

    if len(train_predictions) > 0:
        train_wmae = calculate_wmae(
            np.array(train_actuals),
            np.array(train_predictions),
            np.array(train_holidays).astype(bool)
        )
        print(f"   📈 Training WMAE: ${train_wmae:,.2f}")
        return train_wmae
    else:
        print("   ⚠️ No training predictions available for WMAE calculation")
        return None


In [14]:
def main():
    """Main experiment execution"""
    print("🚀 Starting Experiment SARIMAX: SARIMAX Models using experiment_11_sarimax.ipynb feature engineering")
    print("=" * 80)

    experiment_name = setup_mlflow()

    with mlflow.start_run(run_name="SARIMAX_Walmart_Sales_Complete") as run:
        print(f"🔄 Starting MLflow run: {run.info.run_id}")

        # Log experiment metadata
        mlflow.log_param("experiment_type", "SARIMAX_Individual_Models")
        mlflow.log_param("model_type", "SARIMAX")
        mlflow.log_param("feature_engineering", "experiment_11_sarimax_exact")
        mlflow.log_param("data_split", "temporal_80_20")
        mlflow.log_param("external_regressors", "Yes")
        mlflow.log_param("outlier_removal", "experiment_11_sarimax_exact")
        mlflow.log_param("holiday_weight_evaluation", "5x")
        mlflow.log_param("seasonal_period", "None_for_speed")
        mlflow.log_param("sarimax_order", "(1,1,0)x(0,0,0,0)")
        mlflow.log_param("optimization_method", "lbfgs_ultra_fast")
        mlflow.log_param("min_data_points", "10")
        mlflow.log_param("max_iterations", "10")

        try:
            # Step 1: Get preprocessed data with external features
            print("\n📊 Step 1: Data preprocessing with experiment_11_sarimax features...")
            train_data, val_data, split_info = get_preprocessed_data()

            # Log data info
            mlflow.log_metric("train_samples", len(train_data))
            mlflow.log_metric("val_samples", len(val_data))
            mlflow.log_param("split_date", str(split_info['split_date']))
            mlflow.log_metric("external_features_count", len(get_exog_features()))
            mlflow.log_param("external_features", str(get_exog_features()))

            # Step 2: Prepare SARIMAX data
            print("\n📊 Step 2: Preparing SARIMAX data...")
            train_processed, val_processed, train_combinations, val_combinations = prepare_sarimax_data(train_data, val_data)

            mlflow.log_metric("train_combinations", len(train_combinations))
            mlflow.log_metric("val_combinations", len(val_combinations))
            mlflow.log_metric("missing_combinations", len(train_combinations - val_combinations))

            # Step 3: Train SARIMAX models
            print("\n📈 Step 3: Training SARIMAX models...")
            models, training_errors, model_orders = train_sarimax_models(train_processed, val_processed)

            mlflow.log_metric("successful_models", len(models))

            # Step 4: Make predictions
            print("\n📈 Step 4: Making predictions...")
            y_pred, y_true, is_holiday = make_sarimax_predictions(models, val_processed, train_processed)

            # Step 4.5: Calculate training WMAE
            print("\n📊 Step 4.5: Training performance...")
            train_wmae = calculate_training_wmae(models, train_processed)

            # Step 5: Calculate metrics
            print("\n📊 Step 5: Calculating validation metrics...")

            val_mae = mean_absolute_error(y_true, y_pred)
            val_rmse = np.sqrt(mean_squared_error(y_true, y_pred))
            val_r2 = r2_score(y_true, y_pred)
            val_wmae = calculate_wmae(y_true, y_pred, is_holiday)

            holiday_mask = is_holiday.astype(bool)
            holiday_mae = mean_absolute_error(y_true[holiday_mask], y_pred[holiday_mask]) if holiday_mask.any() else 0
            non_holiday_mae = mean_absolute_error(y_true[~holiday_mask], y_pred[~holiday_mask]) if (~holiday_mask).any() else 0

            # Log all metrics
            mlflow.log_metric("val_wmae", val_wmae)
            mlflow.log_metric("val_mae", val_mae)
            mlflow.log_metric("val_rmse", val_rmse)
            mlflow.log_metric("val_r2", val_r2)
            mlflow.log_metric("holiday_mae", holiday_mae)
            mlflow.log_metric("non_holiday_mae", non_holiday_mae)
            mlflow.log_metric("holiday_samples", int(holiday_mask.sum()))
            mlflow.log_metric("non_holiday_samples", int((~holiday_mask).sum()))

            if train_wmae is not None:
                mlflow.log_metric("train_wmae", train_wmae)

            # Print results
            print("\n" + "=" * 60)
            print("🎯 EXPERIMENT SARIMAX RESULTS SUMMARY")
            print("=" * 60)

            if train_wmae is not None:
                print("📊 Training Metrics:")
                print(f"   Training WMAE: ${train_wmae:,.2f}")
                print()

            print("📊 Validation Metrics:")
            print(f"   WMAE (Competition Metric): ${val_wmae:,.2f}")
            print(f"   MAE: ${val_mae:,.2f}")
            print(f"   RMSE: ${val_rmse:,.2f}")
            print(f"   R²: {val_r2:.4f}")

            print("\n📊 Holiday Breakdown:")
            print(f"   Holiday MAE: ${holiday_mae:,.2f} ({int(holiday_mask.sum())} samples)")
            print(f"   Non-Holiday MAE: ${non_holiday_mae:,.2f} ({int((~holiday_mask).sum())} samples)")

            print("\n📊 Model Statistics:")
            print(f"   Successful models trained: {len(models):,}")
            print(f"   Store-Dept combinations: {len(train_combinations):,}")
            print(f"   External features used: {len(get_exog_features())}")

            # Save model summary
            print("\n💾 Saving model artifacts...")
            model_summary = {
                'total_models': len(models),
                'external_features': get_exog_features(),
                'model_orders': {f"{k[0]}_{k[1]}": v for k, v in model_orders.items()},
                'validation_metrics': {
                    'wmae': val_wmae,
                    'mae': val_mae,
                    'rmse': val_rmse,
                    'r2': val_r2
                }
            }

            import json
            with open('sarimax_model_summary.json', 'w') as f:
                json.dump(model_summary, f, indent=2)
            mlflow.log_artifact('sarimax_model_summary.json')

            print("✅ Experiment SARIMAX completed successfully!")

            experiment = mlflow.get_experiment_by_name(experiment_name)
            if experiment:
                experiment_id = experiment.experiment_id
                run_id = run.info.run_id
                base_url = "https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow"

                print(f"🏃 View run at: {base_url}/#/experiments/{experiment_id}/runs/{run_id}")
                print(f"📊 View experiment at: {base_url}/#/experiments/{experiment_id}")

            print(f"\n🎉 Experiment SARIMAX: Individual SARIMAX Models with experiment_11_sarimax features - COMPLETE!")

        except Exception as e:
            print(f"❌ Experiment failed: {e}")
            mlflow.log_param("error", str(e))
            raise


if __name__ == "__main__":
    main()


🚀 Starting Experiment SARIMAX: SARIMAX Models using experiment_11_sarimax.ipynb feature engineering
🔧 Setting up MLflow and DagsHub...


Initialized MLflow to track repo "konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting"

Repository konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting initialized!

✅ DagsHub initialized successfully!
✅ Created new experiment: Experiment_SARIMAX_20250715_102533
✅ MLflow setup complete!
🔗 Tracking URI: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow
📊 Experiment: Experiment_SARIMAX_20250715_102533
🔄 Starting MLflow run: 416a27d2b0014de5aa6367fb39bee8e1

📊 Step 1: Data preprocessing with experiment_11_sarimax features...
🔄 Getting preprocessed data for SARIMAX...
📊 Loading datasets...
   📈 Train data: (421570, 5)
   🏪 Stores data: (45, 3)
   🎯 Features data: (8190, 12)
   ✅ Merged data: (421570, 17)
   📅 Date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
🧹 Cleaning merged data...
   🔄 Resolving duplicate IsHoliday columns...
   ✅ Cleaned data: (421570, 16) (was (421570, 17))
📅 Creating temporal split (80/19)...
   📊 Split date: 2012-04-13 00:00:00
   📈 Train: 337,256 records (2010-02-05 00:00:00 to 2012-04-13 00:00:00)
   📉 Val: 84,314 records (2012-04-13 00:00:00 to 2012-10-26 00:00:00)
🔧 Fitting preproces